<a href="https://www.kaggle.com/code/khushboos/video-classification-convlstm?scriptVersionId=116849996" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In this code we have used UCF50 data set, where i've considered only the 5 classes from it(because of the space sortage).

# 1. Importing the Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
from tensorflow import keras 
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
#import keras_metrics as km
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
import pickle
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#%cd ../../
#!ls
#%cd ./kaggle/working/
%pwd

'/kaggle/working'

In [3]:
import shutil # To perform the file operation 
data_dir = '../input/ucf50/UCF50' # Original input data directory
os.mkdir('/kaggle/working/UCF50')  # Making my working directory with 5 classes
base_dir = '/kaggle/working/UCF50'
classes = ["BaseballPitch", "CleanAndJerk", "Diving", "Fencing","HighJump"]

"""Copying the 5 classes of the dataset into my working directory."""
for i in classes:
    path = os.path.join(data_dir, i)
    _, dirs, files = next(os.walk(path))
    #print(files)
    for j in files:
        src = os.path.join(path, j)
        dst = os.path.join(base_dir, i)
        
        if not os.path.exists(dst):
            os.mkdir(dst)
            
        shutil.copy(src, dst)

In [4]:
data_dir = '/kaggle/working/UCF50'

# Shape of the image 
img_height , img_width = 64, 64 

# sequencec length of the video frame list
seq_len = 70 

print(len(classes)) 
print(classes)

5
['BaseballPitch', 'CleanAndJerk', 'Diving', 'Fencing', 'HighJump']


In [5]:
#  Creating frames from videos
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
    vidObj.release()
    
    return frames_list

The Above code is to create the data frames from the video using the cv2 library.
>  Videobj = cv2.VideoCapture(video_path)
> > val, image =  Videobj.read()

This will read the video and creates a frame and gives value in binary format, True or False. If the value i strue then we are adding that frame into the frame list of length 70.

In [6]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [7]:
# Test, train directory 
# working_dir = '/kaggle/working'
# data = os.path.join(working_dir, 'X_data')
# label = os.path.join(working_dir, 'Y_label')
# os.mkdir(data)
# os.mkdir(label)
X_data, Y_label = create_data(data_dir)

CleanAndJerk
Defected frame
Fencing
BaseballPitch
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
HighJump
Defected frame
Defected frame
Defected frame
Defected frame
Diving
Defected frame


In [8]:
print("{} : X size, {} : y size\n\n  ".format(len(X_data), len(Y_label)) )

# print("{} : X\n\n{} : Y\n".format(X_data, Y_label))

628 : X size, 628 : y size

  


In [9]:
# Model Creation
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(5, activation = "softmax"))

2023-01-20 06:56:23.276170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 06:56:23.277247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 06:56:23.397940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 06:56:23.398857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-20 06:56:23.399642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [10]:
# X = os.listdir('/kaggle/working/X_data')
# Y =  os.listdir('/kaggle/working/Y_label')
# #print(X)
# print("{} : Length of X\n{} : Length of Y\n".format(len(X), len(Y)))

In [11]:

X_train, X_test, y_train, y_test = train_test_split( X_data, Y_label , test_size=0.20, shuffle=True, random_state=0)

In [12]:
print(" {} : Length of X_train\n {} : Length of Y_train\n {} : Length of X_test\n {} : Length of Y_test\n ".format(len(X_train), len(y_train), len(X_test), len(y_test)))

#print(" {} : X_train\n\n {} : Y_train\n\n {} : X_test\n\n {} : Y_test\n\n ".format(X_train, y_train, X_test, y_test))

 502 : Length of X_train
 502 : Length of Y_train
 126 : Length of X_test
 126 : Length of Y_test
 


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 62, 62, 64)        154624    
_________________________________________________________________
dropout (Dropout)            (None, 62, 62, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 246016)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               62980352  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 63,136,261
Trainable params: 63,136,261
Non-trainable params: 0
____________________________________________

In [14]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [15]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [16]:
history = model.fit( X_train, y_train, epochs=40, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)

with open('history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

2023-01-20 06:56:27.753627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2023-01-20 06:56:31.036198: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


51/51 [==============================] - 62s 1s/step - loss: 1.7340 - accuracy: 0.3691 - val_loss: 1.8268 - val_accuracy: 0.1980
Epoch 2/40
51/51 [==============================] - 53s 1s/step - loss: 1.2398 - accuracy: 0.5187 - val_loss: 2.7519 - val_accuracy: 0.2079
Epoch 3/40
51/51 [==============================] - 53s 1s/step - loss: 0.9238 - accuracy: 0.6783 - val_loss: 1.1112 - val_accuracy: 0.5446
Epoch 4/40
51/51 [==============================] - 53s 1s/step - loss: 0.5156 - accuracy: 0.8254 - val_loss: 3.1595 - val_accuracy: 0.2475
Epoch 5/40
51/51 [==============================] - 53s 1s/step - loss: 0.5003 - accuracy: 0.8454 - val_loss: 0.4577 - val_accuracy: 0.8812
Epoch 6/40
51/51 [==============================] - 54s 1s/step - loss: 0.2521 - accuracy: 0.9501 - val_loss: 0.2944 - val_accuracy: 0.9208
Epoch 7/40
51/51 [==============================] - 53s 1s/step - loss: 0.3006 - accuracy: 0.9127 - val_loss: 0.3163 - val_accuracy: 0.9406
Epoch 8/40
51/51 [=============

In [17]:
y_pred = model.predict(X_test)

In [18]:
# Save the trained model 
# with open('model_pkl', 'wb') as files:
#     pickle.dump(model, files)
# #ConvLSTM = pickle.dump(history)

# #model = pickle.load(ConvLSTM)

# save the model to disk
# import joblib

# filename = 'model.sav'
# joblib.dump(model, filename)

In [19]:
print(y_pred)

[[4.52713517e-04 1.44207831e-02 9.35543776e-02 5.68781495e-01
  3.22790623e-01]
 [9.83773112e-01 3.22468346e-04 7.71784736e-03 6.41040970e-03
  1.77609024e-03]
 [3.09181691e-04 2.81512737e-03 9.27402735e-01 2.97682406e-03
  6.64961487e-02]
 [9.97758746e-01 3.27865244e-04 7.99537083e-05 1.07280466e-04
  1.72609137e-03]
 [1.39179497e-06 2.04604046e-04 1.27141338e-05 7.77947716e-04
  9.99003351e-01]
 [3.97367461e-04 2.84977645e-01 8.88973009e-03 6.15951307e-02
  6.44140065e-01]
 [1.34460555e-04 5.80716645e-04 9.97934699e-01 1.05806964e-03
  2.92087207e-04]
 [3.94143863e-03 9.93180147e-05 9.48279679e-01 4.00452060e-04
  4.72792014e-02]
 [9.99307871e-01 1.64185331e-04 4.31973567e-05 1.84820106e-04
  2.99931446e-04]
 [2.88807962e-04 1.75420882e-05 9.99387145e-01 1.23174788e-04
  1.83390628e-04]
 [2.21895156e-04 1.59689243e-05 9.99419093e-01 1.50247753e-04
  1.92752574e-04]
 [1.62958680e-03 2.62541641e-02 5.92781082e-02 7.65494883e-01
  1.47343248e-01]
 [9.95866656e-01 2.49433774e-03 5.897110

In [20]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)	
# #y_pred = model.predict(X_test)

In [21]:
print(y_pred)

[3 0 2 0 4 4 2 2 0 2 2 3 0 4 4 4 2 4 2 1 0 2 0 0 1 0 4 3 4 4 2 1 1 2 2 2 3
 3 4 1 3 2 4 1 1 4 2 2 2 1 4 2 3 2 4 1 0 2 0 2 0 0 1 4 1 2 0 4 1 4 1 3 0 1
 0 3 2 0 3 0 3 0 2 0 0 1 0 3 4 1 4 3 1 0 4 2 4 1 2 4 4 2 1 4 3 2 1 4 1 0 3
 1 1 0 1 3 2 1 4 2 0 0 3 4 1 0]


In [22]:
print(y_test)

[3 0 2 0 4 1 2 2 0 2 2 3 0 4 3 4 2 4 2 1 0 2 0 0 1 0 4 3 4 4 2 1 1 2 3 2 3
 3 4 1 3 2 4 1 1 4 2 2 2 4 4 2 3 2 4 1 0 3 0 2 0 0 1 2 1 2 0 3 1 4 1 4 0 1
 0 3 2 0 3 0 3 0 2 0 0 1 0 3 4 1 4 3 1 0 4 2 4 1 2 4 4 2 1 4 3 2 1 4 1 0 4
 1 1 0 1 3 2 1 4 2 0 0 3 4 1 0]


In [23]:
from sklearn.metrics import classification_report
import pandas as pd
#print(classification_report(y_test, y_pred))

report_dict = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report_dict)

,0,1,2,3,4,accuracy,macro avg,weighted avg
precision,1.0,0.961538,0.931034,0.882353,0.851852,0.928571,0.925356,0.928427
recall,1.0,0.961538,0.964286,0.789474,0.884615,0.928571,0.919983,0.928571
f1-score,1.0,0.961538,0.947368,0.833333,0.867925,0.928571,0.922033,0.927982
support,27.0,26.000000,28.000000,19.000000,26.000000,0.928571,126.000000,126.000000
